In [5]:
import json
import pandas as pd
# from langdetect import detect
import logging

In [6]:
import spacy
from spacy_fastlang import LanguageDetector
import uk_core_news_trf

# nlp = spacy.load("uk_core_news_trf")
nlp = spacy.load('uk_core_news_sm')
if 'language_detector' in nlp.pipe_names:
    nlp.remove_pipe('language_detector')

# Add the new language detector to the pipeline
language_detector = LanguageDetector()
nlp.add_pipe('language_detector')


In [7]:
view  = []
with open('data/books.jsonlines', 'r') as f:
    for line in f:
        data = json.loads(line)
        description_doc = nlp(data['description'])
        if not data['description'].strip() or sum(c.isalpha() for c in data['description']) < 3: 
            logging.debug(f"Skipping book {data['book_id']} because it has no description") 
            continue
        item = {}
        item['book_id'] = data['book_id']
        item['title'] = data['title']
        item['description'] = " ".join(token.text for token in description_doc if not token.is_punct and not token.is_stop and not token.is_space)
        item['language'] = description_doc._.language

        print(item)            
        view.append(item)

df = pd.DataFrame(view)
df.to_parquet('data/books.parquet', index=False)   

{'book_id': 0, 'title': 'Їсти, молитися, кохати', 'description': "Їсти молитися кохати надзвичайно щира розповідь жінки духовне переродження позитивні зміни житті Автор книги власному прикладі довела найбільших криз відновитися відкрити нові грані можливості рівні переживань Героїня роману письменниця Ліз розчарована сімейному житті зневірена особистому щасті Жінка втрачає головне сенс смак життя Аби набратися сил повернутися життя Ліз вирішує подорожувати шляху країни Італія Індонезія Індія вчиться їсти смаком молитися відкритим серцем кохати наче вперше Популярна письменниця США прославилася світ автобіографічним романом Їсти молитися кохати опублікований 2006 Творчість Елізабет Ґілберт зав'язано життєвому укладі сучасних жінок Любов романтика сімейний побут подорожі вірними супутниками кожної книги автора творах знайти відображення абсолютно жінок дама хоче літератури", 'language': 'uk'}
{'book_id': 1, 'title': 'Серія книжок про Джуді Муді (комплект із 10 книг)', 'description': 'чек

KeyboardInterrupt: 

In [ ]:
# Read the Parquet file
# df = pd.read_parquet('data/books.parquet')
# Print the DataFrame
# print(df)

        book_id                                              title  \
0             0                             Їсти, молитися, кохати   
1             1  Серія книжок про Джуді Муді (комплект із 10 книг)   
2             2                   Три новеллы (комплект из 2 книг)   
3             3  Тренажер мозга. Как развить гибкость мышления ...   
4             4                               У пошуках Івана Сили   
...         ...                                                ...   
241984   241984                               Maps for Lost Lovers   
241985   241985                                    The Round House   
241986   241986                                Diary of a Somebody   
241987   241987                            The Twenty-Seventh City   
241988   241988                                   Children of Ruin   

                                              description  
0       «Їсти, молитися, кохати» - надзвичайно щира ро...  
1       На вас чекає знайомство з кумед

In [ ]:
None >3

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [ ]:
view  = []
with open('data/reviews.jsonlines', 'r') as f:
    for line in f:
        data = json.loads(line)
        description_doc = nlp(data['review'])
        if not data['review'].strip() or sum(c.isalpha() for c in data['review']) < 3: 
            logging.debug(f"Skipping review {data['review']} because it has no content") 
            continue
        item = {}
        item['book_id'] = data['book_id']
        item['review_id'] = data['book_id']
        item['title'] = data['reviewTitle']
        item['is_positive'] = data.get('rating',0) > 3
        item['description'] = " ".join(token.text for token in description_doc if not token.is_punct and not token.is_stop and not token.is_space)
        item['language'] = description_doc._.language

        print(item)            
        view.append(item)

df = pd.DataFrame(view)
df.to_parquet('data/reviews.parquet', index=False)   

{'book_id': 0, 'review_id': 0, 'title': 'Сердечна книга!', 'is_positive': True, 'description': 'Добралася відомого роману вийшов українською мовою Знаєте отримувала такого пристрасного головою зануреного бірюзово океанічного задоволення читання Книга захопила перших сторінок тримала останньої ниткою серця серце близькою розповідь авторки відображає власну історію життя закоханість якою Елізабет простеляє килим речень крізь подорож континентам островам власної душі книга йшла рук Напрочуд легко написана бурхливо перетікає сердечним рікам робота Щиро справжньому відкриті дитячі долоні книга показує життя жінки віддзеркалює мільйоном промінчиків життя кожної Читаючи книгу слова відгукувалися посмішкою грішне смішне сльозами зіжмакане викинуте назавжди зануренням тунель минулого залишеним смарагдом шовковою стежкою майбутнє смолоскипом кричала щасливе живе Безмежно вдячна Елізабет прожите задоволення читачка Захоплююся сим творінням письменниця Сердечно рекомендую читання Ірина Запотоцька'

In [ ]:
import pandas as pd

# Read the Parquet file into a DataFrame
df_reviews = pd.read_parquet('data/reviews.parquet')
positive_reviews = df_reviews[df_reviews['is_positive'] == True].value_counts('book_id').sort_index(ascending=True)
max_reviews = positive_reviews.max()

df_books = pd.read_parquet('data/books.parquet')
df_books = df_books.merge(positive_reviews, on='book_id', how='left')
df_books = df_books.rename(columns={'count': 'positive_reviews_count'})
df_books = df_books.fillna({'positive_reviews_count': 0})
df_books['positive_reviews_score'] = df_books['positive_reviews_count'] / max_reviews

df_books.to_parquet('data/books_enriched.parquet', index=False)   
#todo implement wilson score


In [25]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import spacy
from spacy_fastlang import LanguageDetector
import uk_core_news_trf
import ast


stopwords = []
with open('stopwords_ua_list.txt', 'r') as f:
    stopwords = ast.literal_eval(f.read())
# nlp = spacy.load("uk_core_news_trf")
nlp = spacy.load('uk_core_news_sm')


df_books = pd.read_parquet('data/books_enriched.parquet')
# df_books = df_books[df_books['positive_reviews_score'] > 0]
df_books = df_books.dropna(subset=['description'])
df_books = df_books[df_books['language'] == 'uk']

df_books.to_parquet('data/books_enriched_cleaned.parquet', index=False)   

print(df_books.shape)

# Define a function that will use Spacy to lemmatize the words
def lemmatize_text(text):
    return ' '.join([token.lemma_ for token in nlp(text)])

# Create the TfidfVectorizer with the custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=lemmatize_text, stop_words=stopwords)

tfidf_matrix = vectorizer.fit_transform(df_books['description'])




(89555, 6)


/home/ihor/workspace/nlp-course/lab1_2/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ihor/workspace/nlp-course/lab1_2/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [' ', "'", '-'] not in stop_words.
  warnings.warn(


In [27]:
from scipy.sparse import save_npz

# Save the matrix to a file
save_npz('tfidf_matrix.npz', tfidf_matrix)

In [28]:
from sklearn.metrics.pairwise import pairwise_distances_chunked
import numpy as np
from scipy.sparse import load_npz
from scipy.sparse import save_npz

tfidf_matrix = load_npz('tfidf_matrix.npz')



In [29]:
import pandas as pd

df_books = pd.read_parquet('data/books_enriched.parquet')
# df_books = df_books[df_books['positive_reviews_score'] > 0]
df_boooks = df_books.dropna(subset=['description'])
df_books = df_books[df_books['language'] == 'uk']

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

# Select the first vector
first_vector = tfidf_matrix[0]

# Compute the cosine similarity between the first vector and all others
cosine_similarities = cosine_similarity(first_vector, tfidf_matrix)
print(cosine_similarities)

normalized_scores = df_books['positive_reviews_score'] / df_books['positive_reviews_score'].max()
# Get the cosine similarities of the first vector with all others
first_vector_similarities = cosine_similarities[0] *normalized_scores.values


# Find the index of the vector with the highest cosine similarity to the first vector
# (other than the first vector itself)
most_similar_index = first_vector_similarities.argsort()[-4]
print(first_vector_similarities[most_similar_index])
print(df_books.iloc[most_similar_index])
# print(most_similar_index)


[[1.         0.9963316  0.99262284 ... 0.35132825 0.32018714 0.97889225]]
0.8115768427969166
book_id                                                               51417
title                                                   Чоловік на ім'я Уве
description               Роман Чоловік ім'я Уве літературний дебют попу...
language                                                                 uk
positive_reviews_count                                                 51.0
positive_reviews_score                                                  0.6
Name: 41089, dtype: object
